In [ ]:
import environment
import agent
import pandas as pd
import numpy as np
from tqdm import tqdm
from time import sleep
import torch

train = True
render = False
sub = True
epoch = 1000 if train else 1
agent = agent.DQNAgent(state_size=139, action_size=23, train=True, state_dict=None)
# agent = agent.DQNAgent(state_size=139, action_size=23, train=True, state_dict="weight/save_1.pth")
avg_reward, avg_score = 0, 0

for i in tqdm(range(epoch)):
    env = environment.FactoryEnv()
    
    if sub:
        if i == epoch-1:
            submission = pd.read_csv("data/sample_submission.csv")
            actions = []
            process_n = []        

    for step in range(2184):       
        r = 0
        s = 0
        state = env.get_state()
        mask = agent.get_mask(env)
        action = agent.get_action(state,mask)
        
        if sub:
            if i == epoch-1:
                action_name = env.action_map[action]
                action_process = 0
                if 'PROCESS' in action_name:
                    mylist = action_name.split("_")
                    action_name = mylist[0]
                    action_process = float(mylist[1])                
                if 'CHANGE' in action_name:
                    action_name = "CHANGE_" + str(env.prev_change+1) + str(action-3)
                actions.append(action_name)
                process_n.append(action_process)
      
        if render:
            print('Step : ' + str(env.step_count))
            print('Action : ' + str(env.action_map[action]))
            print('일일 생산량 : ' + str(env.day_process_n))
            print('Score : ' + str(env.prev_score))
            print('')
            
        next_state, reward, done = env.step(action)

        if train:
            agent.append_sample(state, action, reward, next_state, done)
            agent.train_model()
            r += reward 
            s += env.prev_score

        if done:
            break

#     if not train:
#         aaa = list()
#         bbb = list()
#         for j in range(4):
#             env.J[j].save_graph(f"performance/performance{j}.png")
#             info = np.array(env.J[j].info)
#             t, w, o = info[:, 0], info[:, 1], info[:, 2]
#             aaa.append(sum(w)/t[-1])
#             bbb.append(sum(o)/t[-1])
#         print(sum(aaa)/len(aaa))
#         print(sum(bbb)/len(bbb))
    
    if train:
        if i % 10 == 9:
            torch.save(agent.model.state_dict(), f"weight/save_{i + 1}.pth")
                

        avg_reward = r if avg_reward == 0 else avg_reward * 0.9 + r * 0.1
        avg_score = s if avg_score == 0 else avg_score * 0.9 + s * 0.1
#         avg_long_wait = l if avg_long_wait == 0 else avg_long_wait * 0.9 + l * 0.1
        
#         for j in range(4):
#             reward_info[j].append(avg_rewards[j])
#             longest_wait[j].append(avg_long_wait[j])

#         if i % 500 == 499:
#             fig = plt.figure()
#             for j in range(4):
#                 plt.subplot(2, 2, j + 1)
#                 plt.scatter(range(len(reward_info[j])), reward_info[j], s=1)

#                 plt.savefig(f"graph/reward{i + 1}.png")
#             plt.clf()

#             for j in range(4):
#                 plt.subplot(2, 2, j + 1)
#                 plt.scatter(range(len(longest_wait[j])), longest_wait[j], s=1)

#                 plt.savefig(f"graph/longest_wait{i + 1}.png")
#             plt.clf()

#             plt.scatter(range(len(step_info)), step_info, s=1)
#             plt.savefig(f"graph/step{i + 1}.png")
#             plt.clf()
    print("Average Score : " + str(avg_score))
    print("Epoch Score : " + str(env.prev_score))

  0%|                                                                                         | 0/1000 [00:00<?, ?it/s]

Average Score : 24.796333927875
Epoch Score : 24.796333927875


  0%|                                                                               | 1/1000 [00:03<1:02:47,  3.77s/it]

Average Score : 24.578726832523543
Epoch Score : 22.620262974360397


  0%|▏                                                                              | 2/1000 [00:19<2:04:33,  7.49s/it]

...

 55%|█████████████████████████████████████████                                  | 547/1000 [2:36:07<3:03:21, 24.29s/it]

Average Score : 44.443109590088916
Epoch Score : 46.22819279654425


 55%|█████████████████████████████████████████                                  | 548/1000 [2:36:30<3:00:55, 24.02s/it]

Average Score : 44.50379792878404
Epoch Score : 45.04999297704008


 55%|█████████████████████████████████████████▏                                 | 549/1000 [2:36:56<3:05:46, 24.72s/it]

In [ ]:
if sub:
    submission.loc[:, "Event_A"] = actions
    submission.loc[:, "MOL_A"] = process_n
    submission.loc[:, "Event_B"] = actions
    submission.loc[:, "MOL_B"] = process_n

    PRTs = submission[["PRT_1", "PRT_2", "PRT_3", "PRT_4"]].values
    for i in (np.array(env.PRT) - np.array([0,0,554])):
        if i[2] >= 0:
            PRTs[i[2], i[0]] = i[1]
    PRTs = np.ceil(PRTs * 1.1)
    PRTs = PRTs.astype(int)
    submission.loc[:, "PRT_1":"PRT_4"] = PRTs
    submission.to_csv(f"submission/save_{epoch}.csv", index=False)

In [ ]:
env.p

In [ ]:
env.q

In [ ]:
env.c_t

In [ ]:
env.c_n

In [ ]:
env.s_t

In [ ]:
env.s_n

In [ ]:
a = np.array([1,2,3])

In [ ]:
a.astype(float)